In [6]:
# import library
from bs4 import BeautifulSoup
import requests
from collections import deque
import time
import numpy as np
import json
import os

Collect and save STEAM STORE data in json format with the duration of multiple years
Specify the path to load appid

In [4]:
# variable Z from 
Z = ["required_age","is_free","controller_support","dlc","metacritic","platforms","recommendations","achievements","release_date"]

def please(id,idx,res):
    g_info = {}
    try:
        data = res.json()
        if data[str(id)]["success"] == True: # success == true
            keys = data[str(id)]["data"].keys()
            main = data[str(id)]["data"]
            
            for elem in Z:
                if elem in keys:
                    if elem in ["recommendations","achievements"]:
                        g_info[elem] = main[elem]["total"]
                    elif elem == "release_date":
                        g_info[elem] = main[elem]["date"]
                    elif elem == "metacritic":
                        g_info[elem] = main[elem]["score"]
                    elif elem == "dlc":
                        g_info[elem] = len(main[elem])
                    else:
                        g_info[elem] = main[elem]
                else:
                    g_info[elem] = np.nan
            print(idx,id,g_info)
        else: # success == false
            for elem in Z:
                g_info[elem] = np.nan
            print("error",idx,id,g_info)
                
    except: # when empty page is loaded - 
        for elem in Z:
            g_info[elem] = np.nan
        print("error",idx,id,g_info)
            
    finally:
        final_data[str(id)] = g_info

In [ ]:
for year in [2019,2018]:
    cwd = os.getcwd()
    parent_directory = os.path.join(cwd, os.pardir)

    # relative_path
    relative_path = f'CRAWLING DATA\\appid\\appid_{year}.txt'

    # absolute_apath
    absolute_path = os.path.join(parent_directory, relative_path)
    file = open(absolute_path,"r")
    final_data = {}

    appids = []
    while True:
        line = file.readline()
        if not line: break
        appids.append(int(line.strip()))
    
    for idx,id in enumerate(appids):
        path = f"https://store.steampowered.com/api/appdetails?appids={id}"

        res = requests.get(path)
        if res.status_code == 200:
            please(id,idx,res)
        else:
            time.sleep(60)
            please(id,idx,res)
        time.sleep(1.5)
    # print(final_data)

    save_path = f"steam_store_data_{year}.json"
    with open(save_path,'w') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=4)